# Noyau Sémantique

Dans cet exemple de code, vous utiliserez le framework d'IA [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) pour créer un agent basique.

L'objectif de cet exemple est de vous montrer les étapes que nous utiliserons plus tard dans les exemples de code supplémentaires lors de la mise en œuvre des différents modèles d'agents.


## Importez les packages Python nécessaires


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Création du client

Dans cet exemple, nous utiliserons [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) pour accéder au LLM.

L'`ai_model_id` est défini comme `gpt-4o-mini`. Essayez de changer le modèle pour un autre disponible sur le marketplace de GitHub Models afin de voir les différents résultats.

Pour utiliser le `Azure Inference SDK` qui est utilisé pour le `base_url` des GitHub Models, nous utiliserons le connecteur `OpenAIChatCompletion` au sein de Semantic Kernel. Il existe également d'autres [connecteurs disponibles](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) pour utiliser Semantic Kernel avec d'autres fournisseurs de modèles.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Création de l'Agent

Ci-dessous, nous créons l'Agent appelé `TravelAgent`.

Pour cet exemple, nous utilisons des instructions très simples. Vous pouvez modifier ces instructions pour voir comment l'agent réagit différemment.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Exécution de l'Agent

Nous pouvons maintenant exécuter l'Agent en définissant un thread de type `ChatHistoryAgentThread`. Tous les messages système nécessaires sont fournis à l'argument clé `messages` de la méthode invoke_stream de l'agent.

Une fois ces éléments définis, nous créons un `user_inputs` qui correspondra à ce que l'utilisateur envoie à l'agent. Dans ce cas, nous avons défini ce message comme étant `Plan me a sunny vacation`.

N'hésitez pas à modifier ce message pour voir comment l'agent réagit différemment.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
